# i.MX93 EdgeLock 2GO Provisioning - Indirect flow

This notebook demonstrates EdgeLock 2GO (EL2GO) provisioning on i.MX93 using a two-step approach:
1. Get Secure Objects from EdgeLock 2GO service
2. Provision the Secure Objects to the target

This approach provides more flexibility compared to the single-shot method, allowing you to:
- Download Secure Objects once and provision them to multiple devices
- Provision devices that might be offline later
- Modify or inspect Secure Objects before provisioning

## Overview of the Provisioning Process

The provisioning process involves the following key components:

- **SPSDK**
- **OEM Provisioning Application**

## OEM Provisioning Application Overview

The **OEM Provisioning Application** is available for download from [GitHub](https://github.com/nxp-imx/oem-prov-app).

This application also includes a Yocto layer that provides the following capabilities:
- Integrates the **OEM Provisioning Application** into the generated root filesystem.
- Adds the **OEM Provisioning Application configuration file** to the generated root filesystem.
- Configures the application to run automatically at boot time.
- Configures the **CONFIG_CONSOLE_MUX** setting for U-Boot.

For further details about the Yocto layer, please refer to the [OEM Provisioning Application Documentation](https://github.com/nxp-imx/oem-prov-app).

The **OEM Provisioning Application** is a reference application designed to perform OEM provisioning. This process involves importing security assets, formatted by the **EdgeLock 2GO server**, into the **EdgeLock Enclave**. OEM provisioning is critical for ensuring device security and effective identity management.

For comprehensive information on the **OEM Provisioning Application** and its supported modes of operation, please consult the [application documentation](https://github.com/nxp-imx/oem-prov-app). For this demonstration, the application is configured to run in the indirect flow mode.

## SPSDK and OEM Provisioning Application Workflow

1. **Integrate the OEM Provisioning Application**
* Integrate the application into the i.MX Linux OS BSP and configure it to run at boot time.
* For detailed instructions refer to [OEM Provisioning Application Documentation](https://github.com/nxp-imx/oem-prov-app).
2. **Build the i.MX Linux OS BSP** 
* Build the i.MX Linux OS BSP and write the resulting images to the eMMC. 
* Further guidance is available in the **i.MX Linux OS BSP documentation**.
3. **Download the security assets**
* Use **SPSDK** to download the security assets from the **EdgeLock 2GO server**.
4. **Write the security assets to a FAT32 partition**
* The user can either:
    * Partition the eMMC/SD card according to particular needs
    * Use the image generated by the **i.MX Linux OS BSP**, which already contains partitions. The **i.MX Linux OS BSP** build environment generates a SD card image (wic) that includes a VFAT partition used for storing the kernel image and DTB file. This partition can be used for storing the security assets. For more details about the generated wic image, refer to **i.MX Linux OS BSP** documentation.
5. **Boot the Linux system**
* Boot the system to begin the provisioning process
6. **Provisioning status**
* Upon boot, the **OEM Provisioning Application** will execute automatically and provision the security assets. 
* To verify the status of the provisioning, use the following command:

   ```bash
   systemctl status oem-prov


## SPSDK Role and Workflow

In the EdgeLock 2GO provisioning process for i.MX93, SPSDK serves as the critical intermediary between the EdgeLock 2GO service and the target device. Here's how the workflow operates:

1. **Bootloader Download**: 
   - SPSDK downloads the U-Boot bootloader to the target device using the SDPS interface
   - This establishes a base execution environment for further operations
   - U-Boot bootloader must be built with CONFIG_CONSOLE_MUX=y flag.
   - If you use the OEM Provisioning Application Yocto layer, the generated u-boot has already the above mentioned define set.

2. **Device Identification**:
   - SPSDK communicates with the device to obtain its UUID (Universally Unique Identifier)
   - This UUID is the device's digital fingerprint that uniquely identifies it in the EdgeLock 2GO ecosystem

3. **Asset Retrieval**:
   - Using the device's UUID, SPSDK queries the EdgeLock 2GO server to download the device-specific provisioning assets
   - These assets include secure objects, certificates, keys, and other security elements

4. **Storage Preparation**:
   - SPSDK writes the downloaded assets to a FAT32/VFAT partition on the device
   - This partition is accessible from the Linux environment that runs the OEM Provisioning Application
   - This approach ensures the assets are available for the OEM Provisioning Application to import into the EdgeLock Enclave
   - This partition should be FAT32/VFAT because SPSDK is using u-boot to write the assets into the partition. U-boot cannot access other partition types.
   - As mentioned before the user can use the boot partition (partition used for storing the kernel and the dtb file) from the wic image generated by the i.MX Linux OS BSP build system. Alternatively, the user can partition the device (eMMC/SD card) per particular needs and format the partition as FAT32/VFAT.

5. **Optional Boot Sequence**:
   - After downloading and storing the provisioning assets, SPSDK can optionally boot Linux
   - This allows for immediate provisioning without requiring manual intervention

This workflow demonstrates SPSDK's versatility as a bridge between the cloud-based EdgeLock 2GO service and the physical hardware, handling both communication protocols and asset management within the provisioning pipeline.




## 1. Prerequisites
- EdgeLock 2GO account and API token defined. For details about generating an API token refer to the EdgeLock 2GO documentation.
- i.MX93 board
- i.MX Linux OS BSP (including O**EM Provisioning Application**) **images already installed on the device** - UUU or SPSDK nxpuuu application might be used for downloading the image to SD/EMMC.
- U-Boot bootloader with CONFIG_CONSOLE_MUX=y enabled. This config allows SPSDK to be able to interact with the U-boot console in order to get UUID and to write securiry assets.

## 2. Setup

In [6]:
# Initialization cell
from spsdk.utils.jupyter_utils import YamlDiffWidget

# setup this jupyter notebook
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


### 2.1 Configuration Setup

First we'll create a configuration file containing all necessary information for i.MX93.
To generate a configuration template use `el2go-host get-template` command

In [7]:
YamlDiffWidget("el2go_config.diffc").html

el2go-host get-template -f mimx9352 -o config_template.yaml --force
The EL2GO template for mimx9352 has been saved into config_template.yaml YAML file


### 2.2 Edit the Configuration File

Now, edit the configuration file to include your specific settings:

1. API key: Use path to a file (e.g., ~/.el2go/qa_token.txt) or environment variable
2. Device Group ID: From your EdgeLock 2GO account
3. Product 12NC: From your EdgeLock 2GO account
4. Secure Objects address: Address in RAM for storing external secure objects before writing them to FAT partition
5. Specify configuration for fatwrite parameters
6. The OEM Provisioning Configuration file can be optionally loaded to the FAT32 partition if the user infrastructure requires such a scenario. For this demonstration the application configuration file is copied to the root filesystem by the i.MX Linux OS Yocto infrastructure (after applying the OEM Provisioning Application Yocto layer).

### 2.3 HW Setup

Connect the target board (i.MX93 EVK) via USB in serial downloader mode and run device discovery 
You may use `nxpdevscan` utility to list connected devices.

In [3]:
%! nxpdevscan --uuu

nxpdevscan --uuu 
-------- Connected NXP UUU Devices --------

Path: 3:2332
Chip: MX93
Product: SDPS:
Vendor ID: 0x1fc9
Product ID: 0x014e
BCD: 1
Serial Number: DC193F680C2142FF



## 3. Two-Step EL2GO Provisioning

### 3.1 Step 1: Get Secure Objects

First, we'll get the secure objects from EdgeLock 2GO service and save them to a file.
This step requires internet connectivity but can be done once for multiple devices.

In [8]:
%! el2go-host get-secure-objects -f mimx9352 --force --config config_example.yaml --output output/secure_objects_open.bin --re-assign

el2go-host get-secure-objects -f mimx9352 --force --config config_example.yaml --output output/secure_objects_open.bin --re-assign 
SDPS[-t 10000]: boot -f imx-boot-imx93-9x9-lpddr4-qsb-sd.bin-flash_singleboot


Secure Objects stored to output/secure_objects_open.bin


### 3.2 Step 2: Provision Objects

Now that we have the secure objects saved in a file, we can provision them to the target.
This step requires the target to be connected but doesn't need internet connectivity.

In [9]:
%! el2go-host provision-objects -f mimx9352 --config config_example.yaml --secure-objects-file output/secure_objects_open.bin

el2go-host provision-objects -f mimx9352 --config config_example.yaml --secure-objects-file output/secure_objects_open.bin 
Writing Secure Objects to MMC/SD FAT: assets.bin
Data written 129 bytes written in 2 ms (62.5 KiB/s)
Detect USB boot. Will enter fastboot mode!

Writing OEM Provisioning Config to MMC/SD FAT: config.yaml
Data written 245 bytes written in 9 ms (26.4 KiB/s)
Detect USB boot. Will enter fastboot mode!

Booting Linux
  Command: setenv mmcdev 0 -> Detect USB boot. Will enter fastboot mode!

  Command: setenv mmcpart 1 -> Detect USB boot. Will enter fastboot mode!

  Command: setenv mmcroot /dev/mmcblk0p2 rootwait rw -> Detect USB boot. Will enter fastboot mode!

  Command: run loadimage -> 34912768 bytes read in 149 ms (223.5 MiB/s)
Detect USB boot. Will enter fastboot mode!

  Command: run loadfdt -> 43703 bytes read in 22 ms (1.9 MiB/s)
Detect USB boot. Will enter fastboot mode!

  Command: run bsp_bootcmd -> 
Secure Objects uploaded successfully


## 4. Alternative: Single-Shot Provisioning

Alternatively, you can perform both steps in a single command using the `provision-device` command.
This approach is simpler but requires internet connectivity at the time of provisioning.

In [14]:
%! el2go-host provision-device -f mimx9352 --config config_example.yaml 

el2go-host provision-device -f mimx9352 --config config_example.yaml 
SDPS[-t 10000]: boot -f imx-boot-imx93-9x9-lpddr4-qsb-sd.bin-flash_singleboot


Writing Secure Objects to MMC/SD FAT: assets.bin
Data written 129 bytes written in 7 ms (17.6 KiB/s)
Detect USB boot. Will enter fastboot mode!

Writing OEM Provisioning Config to MMC/SD FAT: config.yaml
Data written 245 bytes written in 9 ms (26.4 KiB/s)
Detect USB boot. Will enter fastboot mode!

Booting Linux
  Command: setenv mmcdev 0 -> Detect USB boot. Will enter fastboot mode!

  Command: setenv mmcpart 1 -> Detect USB boot. Will enter fastboot mode!

  Command: setenv mmcroot /dev/mmcblk0p2 rootwait rw -> Detect USB boot. Will enter fastboot mode!

  Command: run loadimage -> 34912768 bytes read in 218 ms (152.7 MiB/s)
Detect USB boot. Will enter fastboot mode!

  Command: run loadfdt -> 43703 bytes read in 22 ms (1.9 MiB/s)
Detect USB boot. Will enter fastboot mode!

  Command: run bsp_bootcmd -> 
Secure Objects uploaded successf

## 6. Summary

This notebook demonstrated two approaches for EdgeLock 2GO provisioning on i.MX93:

1. **Two-Step Approach**:
   - Get Secure Objects (`get-secure-objects`)
   - Provision Objects (`provision-objects`)

2. **Single-Shot Approach**:
   - Provision Device (`provision-device`)

The two-step approach offers more flexibility, especially in production environments where you might want to:
- Download Secure Objects once and provision multiple devices
- Provision devices in environments without internet connectivity
- Validate or modify Secure Objects before provisioning

The single-shot approach is simpler for quick testing and development purposes.